In [2]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI  # 나중에 모델을 바꿀때 이부분만 변경하면 ok

# default text-davinci-003
# llm = OpenAI()
# default gpt-3.5-turbo
chat = ChatOpenAI(
    temperature=0.1  # 0.1 -> 덜 창의적 0.9 -> 창의적 올라가고 무작위성도 올라감
)

# 이렇게 함수 통합 되어있는거 볼 수 있음
# llm.predict("물어볼꺼")
# chat.predict("물어볼꺼")

In [3]:
# message constuructor
# HumanMessage -> 사람이 보내는 메세지
# AIMessage -> AI 메세지
# SystemMessage -> AI의 기본 설정, 기본 값, 기본 context 지정 하는데 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Korean",
    ),
    AIMessage(content="Ciao, mi chimo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?",
    ),
]

# chat.predict_messages(messages)

In [4]:
# ChatPromptTemplate -> template을 message로 부터 만듬
# PromptTemplate -> 그냥 String 을 통해서 만듬
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# {} -> 일종의 placeholder
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}.",
)

chat = ChatOpenAI(temperature=0.1)

prompt = template.format(country_a="Korea", country_b="Japan")

# chat.predict(prompt)

In [18]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Korea",
    name="Soctates",
    country_a="Korea",
    country_b="Japan",
)

chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 제 이름은 AI입니다. 한국과 일본 사이의 거리는 대략 1,000km 정도입니다.')

In [7]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")  # 공백 제거 + string split array
        return list(map(str.strip, items))


p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [9]:
template = ChatPromptTemplate.from_messages([
        ("system","You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do Not reply with anything else."),
        ("human", "{question}"),
])

prompt = template.format_messages(max_items=10 , question = "Battle.net 에는 어떤 게임이 있어?")

result = chat.predict_messages(prompt)
print(result)

p = CommaOutputParser()

p.parse(result.content)

content='overwatch, world of warcraft, hearthstone, diablo, starcraft, heroes of the storm, warcraft iii, call of duty, destiny, diablo ii'


['overwatch',
 'world of warcraft',
 'hearthstone',
 'diablo',
 'starcraft',
 'heroes of the storm',
 'warcraft iii',
 'call of duty',
 'destiny',
 'diablo ii']

In [10]:
# chain
# 모든 요소를 하나로 합치는 것
# 합쳐진 요소들은 하나의 chain으로 실행됨 하나하나 순서대로 result를 반환할때 까지

# | <- langchain
# Template , Language Model , Output Parser 으로 구성된 chain
# 즉 반환값이 -> 방향으로 전달되는 것
# !구조채로 선언된 tempalte 을 chat 모델에 넘겨주고 그걸 outputparser에게 넘겨주는 체인
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 2,
    "question": "What are the hoshimachi suisei songs?"
})

['stella', 'next color planet']

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# callbacks -> 볼 수 있는 문자가 생길 때 마다 print 를 console에 해줌
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])
# 이 경우에는 parser 필요 x
# 두가지의 chef 생성
# 1. 일반적인 요리를 하는 chef
# 2. 채식 주의자 요리로 변환해주는 chef
chef_chain = chef_prompt | chat

In [20]:
# 비건 prompt
# 채식 주의자를 위한 prompt
# 전통적인 채식 주의자용 레시피 , 레시피 입력 받고 채식 주의자 용으로 변횐
# 대체 재료 찾고 , 준비하는 방법 제공
# 기존 레시피 변경 많이 x
# 대체품 x -> 레시피 모른다 말하기 ( 중요 )
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alterative ingredients and explain their preparation. Tou don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace"),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

In [22]:
# 일반적인 레시피를 채식 주의자용 레시피로 변환!
# {"recipe": chef_chain} -> langChain 언어라고 보면됨 
# RunnableMap 이라고 함
# 코드 실행하고 응답결과를 다음으로 넘기는 방식으로 사용가능
# 출력값을 다음 chain으로 넘긴다 이해
# chef_chain.invoke() 이렇게 해서 값을 넘긴다고 생각하면 좋음
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian",
})

인도 음식을 요리하고 싶으시군요! 인도 음식은 향신료와 다양한 맛으로 유명합니다. 여기 간단하면서도 맛있는 인도 요리 레시피를 알려드리겠습니다. 오늘은 "버터 치킨"을 만들어 보는 건 어떠신가요?

재료:
- 닭 가슴살 500g
- 요거트 1컵
- 레몬즙 2큰술
- 생강 약간
- 마늘 4쪽
- 향신료 (계피, 카르다몬, 클로브) 약간
- 코리앤더 파우더 1큰술
- 케이푸트 1큰술
- 버터 50g
- 토마토 소스 1컵
- 신선한 크림 1/2컵
- 소금과 후추 약간
- 신선한 코리앤더 (장식용)

요리 순서:
1. 닭 가슴살을 세로로 반으로 잘라 작은 조각으로 만듭니다.
2. 그릇에 요거트, 레몬즙, 다진 생강, 다진 마늘, 향신료, 코리앤더 파우더, 케이푸트를 넣고 잘 섞어줍니다.
3. 닭 조각을 이 소스에 넣고 잘 버무려 줍니다. 최소 1시간 동안 재워주세요.
4. 팬에 버터를 녹여 닭 조각을 구워줍니다. 닭이 완전히 익을 때까지 약 10-15분 정도 구워주세요.
5. 다른 팬에 토마토 소스를 데치고 신선한 크림을 넣어줍니다. 소금과 후추로 간을 맞춰주세요.
6. 구워진 닭 조각을 토마토 소스에 넣고 약 5분 정도 더 끓여줍니다.
7. 그릇에 닭 조각을 담고 신선한 코리앤더로 장식해주세요.
8. 밥이나 나안, 로티와 함께 즐겨보세요!

이제 인도 버터 치킨을 즐길 준비가 되었습니다. 맛있게 드세요!인도 버터 치킨은 맛있는 요리입니다! 하지만 닭 가슴살을 대체할 수 있는 채식 재료는 없습니다. 그러나, 닭 가슴살 대신 대체 육류를 사용할 수 있습니다. 대체 육류는 대부분 콩, 대두, 마이토, 채소 등으로 만들어진 식물성 육류입니다. 이러한 대체 육류는 대부분 슈퍼마켓이나 온라인에서 구매할 수 있습니다. 대체 육류를 사용하면 버터 치킨의 맛과 질감을 유지하면서도 채식 옵션을 제공할 수 있습니다.

또한, 버터를 대체할 수 있는 옵션으로는 식물성 버터나 마가린을 사용할 수 있습니다. 식물성 버터는 대부분 옥수수 오일, 코코넛 오일, 팜 오일 등을 기반으로 만들어진 

AIMessageChunk(content='인도 버터 치킨은 맛있는 요리입니다! 하지만 닭 가슴살을 대체할 수 있는 채식 재료는 없습니다. 그러나, 닭 가슴살 대신 대체 육류를 사용할 수 있습니다. 대체 육류는 대부분 콩, 대두, 마이토, 채소 등으로 만들어진 식물성 육류입니다. 이러한 대체 육류는 대부분 슈퍼마켓이나 온라인에서 구매할 수 있습니다. 대체 육류를 사용하면 버터 치킨의 맛과 질감을 유지하면서도 채식 옵션을 제공할 수 있습니다.\n\n또한, 버터를 대체할 수 있는 옵션으로는 식물성 버터나 마가린을 사용할 수 있습니다. 식물성 버터는 대부분 옥수수 오일, 코코넛 오일, 팜 오일 등을 기반으로 만들어진 제품입니다. 마가린은 대부분 식물성 오일과 물, 소금 등을 혼합하여 만들어진 제품입니다. 이러한 대체 버터를 사용하면 버터 치킨의 풍미를 유지하면서도 채식 옵션을 제공할 수 있습니다.\n\n마지막으로, 신선한 크림을 대체할 수 있는 옵션으로는 식물성 크림을 사용할 수 있습니다. 식물성 크림은 대부분 콩, 대두, 코코넛 등을 기반으로 만들어진 제품입니다. 이러한 대체 크림을 사용하면 버터 치킨의 부드러운 텍스처와 크림 소스의 맛을 유지하면서도 채식 옵션을 제공할 수 있습니다.\n\n인도 버터 치킨은 매우 맛있는 요리이지만, 일부 재료는 대체하기 어렵습니다. 그러나 대체 육류, 식물성 버터 또는 마가린, 식물성 크림을 사용하여 채식 옵션을 만들 수 있습니다. 이러한 대체 재료를 사용하여 버터 치킨을 즐겨보세요!')